In [ ]:
import numpy as np
import pandas as pd

In [ ]:
train_df=pd.read_csv('../input/hackerearths-snakes-in-the-hood/dataset/train.csv')

In [ ]:
train_df.head()

In [ ]:
for i in range(len(train_df['image_id'])):
    train_df['image_id'][i]=train_df['image_id'][i]+'.jpg'
    
    
    

In [ ]:
len(train_df['image_id'])

In [ ]:
train_df.head()

In [ ]:
train_df.tail()

In [ ]:
from keras.preprocessing import image

In [ ]:
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input,decode_predictions
from keras.layers import Input, Conv2D, MaxPool2D, Dense, Flatten
from keras.models import Model
from keras.utils import to_categorical

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255,)
        # shear_range=0.2,
        # zoom_range=0.2,
        # horizontal_flip=True,
        # width_shift_range=0.1,
        # height_shift_range=0.1

train_generator=train_datagen.flow_from_dataframe(dataframe=train_df,
                                            directory="../input/hackerearths-snakes-in-the-hood/dataset/train/",
                                            x_col="image_id",
                                            y_col="breed",
                                            subset="training",
                                            batch_size=128,
                                            seed=42,
                                            shuffle=True,
                                            class_mode="categorical",
                                            target_size=(299,299))

In [ ]:
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.resnet50 import preprocess_input,decode_predictions
from keras.preprocessing import image

In [ ]:
resnet=InceptionResNetV2(include_top=True,weights='imagenet')

In [ ]:
x=resnet.layers[-2].output
fc1=Dense(35,activation='softmax')(x)

In [ ]:
my_model=Model(inputs=resnet.input,outputs=fc1)

In [ ]:
from keras.optimizers import Adam

In [ ]:
adam=Adam(learning_rate=0.001)

In [ ]:
for l in my_model.layers[:-35]:
    #print(l)
    l.trainable = False
my_model.compile(optimizer='adam',loss ="categorical_crossentropy",metrics=["accuracy"])

In [ ]:
# my_model.fit_generator(train_generator,epochs=3,steps_per_epoch=5508//128)

In [ ]:
my_model.save('model.h5')

In [ ]:
test_df=pd.read_csv('../input/hackerearths-snakes-in-the-hood/dataset/test.csv')

In [ ]:
for i in range(len(test_df['image_id'])):
    test_df['image_id'][i]=test_df['image_id'][i]+'.jpg'

In [ ]:
test_df.head()

In [ ]:
test_df.shape

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255,)
        # shear_range=0.2,
        # zoom_range=0.2,
        # horizontal_flip=True,
        # width_shift_range=0.1,
        # height_shift_range=0.1

test_generator=test_datagen.flow_from_dataframe(dataframe=test_df,
                                            directory="../input/hackerearths-snakes-in-the-hood/dataset/test/",
                                            x_col="image_id",
                                            y_col=None,
                                            batch_size=128,
                                            seed=42,
                                            shuffle=False,
                                            class_mode=None,
                                            target_size=(331,331))

In [ ]:
y_pred=my_model.predict_generator(test_generator,
verbose=1)

In [ ]:
y_pred.shape

In [ ]:
y_pred_2=np.argmax(y_pred,axis=1)

In [ ]:
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in y_pred_2]

In [ ]:
predictions

In [ ]:
test_df=pd.read_csv('../input/hackerearths-snakes-in-the-hood/dataset/test.csv')

In [ ]:
test_df.head()

In [ ]:
test_name=test_df['image_id'].values

In [ ]:
test_name.shape

In [ ]:
data=pd.DataFrame((zip(test_name,predictions)),columns=['image_id','breed'])

In [ ]:
data.to_csv('output.csv',index=False)

In [ ]:
x_test=[]

In [ ]:
img=image.load_img('../input/hackerearths-snakes-in-the-hood/dataset/test/00a9bba384.jpg',target_size=(331,331))

In [ ]:
img=image.img_to_array(img)/255

In [ ]:
x_test.append(img)

In [ ]:
x_test=np.array(x_test)

In [ ]:
pre=my_model.predict(x_test)

In [ ]:
pre=np.argmax(pre)

In [ ]:
pre